In [ ]:
!pip install gensim pandas openpyxl

In [ ]:
# Import library
import pandas as pd
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk
import numpy as np

In [ ]:
# Download tokenizer (jika belum)
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Baca file Excel
df = pd.read_excel('/content/combined_preprocessed.xlsx')  # Ganti dengan nama file kamu
# Contoh melihat 5 baris pertama
print(df.head())


                                               Title  \
0  Efek Ekor Jas Pencalonan Anies, Elektabilitas ...   
1  Ekonomi 2024 Ditargetkan Tumbuh 5,7 Persen, pa...   
2  Survei Litbang Kompas: PDI-P, Gerindra, dan Go...   
3  Survei Litbang "Kompas": Popularitas Golkar Te...   
4  "Endorsement" dan Basa-basi Politik ala Jokowi...   

                            Timestamp  \
0         21 Februari 2023, 15:30 WIB   
1  Kompas.com - 21/02/2023, 14:22 WIB   
2         21 Februari 2023, 09:58 WIB   
3  Kompas.com - 21/02/2023, 05:23 WIB   
4  Kompas.com - 21/02/2023, 05:20 WIB   

                                            FullText  \
0  Hasil jajak pendapat yang diselenggarakan Litb...   
1  JAKARTA, KOMPAS.com - Pemerintah menargetkan p...   
2  PDI-Perjuangan, Partai Gerindra, dan Partai Go...   
3  JAKARTA, KOMPAS.com - Survei Litbang Kompas Ja...   
4   JAKARTA, KOMPAS.com - Presiden Joko Widodo la...   

                                         CleanNarasi  \
0  Efek Ekor Jas Pencal

In [ ]:
# 1. Cek total NaN di kolom 'label'
total_nan = df['label'].isna().sum()
print(f"\nTotal NaN di kolom 'label': {total_nan}")

# 2. Lihat isi data (baris) yang memiliki NaN di kolom 'label'
rows_with_nan = df[df['label'].isna()]
print("\nBaris dengan NaN di kolom 'label':")
print(rows_with_nan)


Total NaN di kolom 'label': 21

Baris dengan NaN di kolom 'label':
     Title Timestamp FullText CleanNarasi  Url  label sumber scrap  \
2295   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
2418   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
3770   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
3962   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
3994   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
4049   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
4050   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
4246   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
4247   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
4248   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
4249   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
4312   NaN       NaN      NaN         NaN  NaN    NaN    NaN   NaN   
4314   NaN       NaN  

In [ ]:
# 3. Hapus baris dengan NaN di kolom 'label'
df_cleaned = df.dropna(subset=['label'])

print("\nDataFrame setelah menghapus baris dengan NaN di kolom 'label':")
print(df_cleaned)


DataFrame setelah menghapus baris dengan NaN di kolom 'label':
                                                   Title  \
0      Efek Ekor Jas Pencalonan Anies, Elektabilitas ...   
1      Ekonomi 2024 Ditargetkan Tumbuh 5,7 Persen, pa...   
2      Survei Litbang Kompas: PDI-P, Gerindra, dan Go...   
3      Survei Litbang "Kompas": Popularitas Golkar Te...   
4      "Endorsement" dan Basa-basi Politik ala Jokowi...   
...                                                  ...   
19029  (HOAX) Foto Raja Salman Mengangkat Tandu Jenaz...   
19030  (HOAX) Jokowi dituduh bodoh, salah cara dalam ...   
19031  (FITNAH) Jokowi dituduh memasukkan ribuan tena...   
19032                   (HOAX) Mata Uang Rupiah Terpuruk   
19033  (FITNAH) Ahok izinkan Natalan di Monas – tapi ...   

                                Timestamp  \
0             21 Februari 2023, 15:30 WIB   
1      Kompas.com - 21/02/2023, 14:22 WIB   
2             21 Februari 2023, 09:58 WIB   
3      Kompas.com - 21/02/2023, 05:

In [ ]:
# Latih model Word2Vec CBOW dan Skip-gram
print("\nMelatih model CBOW...")
model_cbow = Word2Vec(
    sentences=df_cleaned['tokens_no_stopwords'],
    vector_size=100,  # Ukuran vektor
    window=5,         # Jumlah kata di sekitar
    min_count=1,      # Frekuensi minimum kata
    workers=4,        # Jumlah thread
    sg=0              # 0: CBOW
)

print("Melatih model Skip-gram...")
model_sg = Word2Vec(
    sentences=df_cleaned['tokens_no_stopwords'],
    vector_size=100,  # Ukuran vektor harus sama untuk penggabungan
    window=5,         # Jumlah kata di sekitar
    min_count=1,      # Frekuensi minimum kata
    workers=4,        # Jumlah thread
    sg=1              # 1: Skip-gram
)


Melatih model CBOW...


Melatih model Skip-gram...


In [ ]:
# Simpan model (opsional)
model_cbow.save("word2vec_model_cbow.model")
model_sg.save("word2vec_model_sg.model")

In [ ]:
# Fungsi untuk mendapatkan vektor rata-rata dari sebuah kalimat untuk model tertentu
def get_sentence_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# --- Bagian Baru: Dapatkan vektor dari CBOW, Skip-gram, dan Gabungan ---
print("\nMenghitung vektor kalimat...")


Menghitung vektor kalimat...


In [ ]:
# Vektor CBOW
df_cleaned['vector_cbow'] = df_cleaned['tokens_no_stopwords'].apply(lambda x: get_sentence_vector(x, model_cbow))

# Vektor Skip-gram
df_cleaned['vector_sg'] = df_cleaned['tokens_no_stopwords'].apply(lambda x: get_sentence_vector(x, model_sg))

# Vektor Gabungan (rata-rata dari CBOW dan Skip-gram)
df_cleaned['vector_combined'] = df_cleaned.apply(
    lambda row: (np.array(row['vector_cbow']) + np.array(row['vector_sg'])) / 2.0,
    axis=1
)

/tmp/ipython-input-4102254164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['vector_cbow'] = df_cleaned['tokens_no_stopwords'].apply(lambda x: get_sentence_vector(x, model_cbow))
/tmp/ipython-input-4102254164.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['vector_sg'] = df_cleaned['tokens_no_stopwords'].apply(lambda x: get_sentence_vector(x, model_sg))
/tmp/ipython-input-4102254164.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [ ]:
# Lihat hasil
print("\nHasil vektor untuk beberapa baris:")
print(df_cleaned[['CleanedNarasi', 'vector_cbow', 'vector_sg', 'vector_combined']].head())


Hasil vektor untuk beberapa baris:
                                       CleanedNarasi  \
0  efek ekor jas pencalonan anies  elektabilitas ...   
1  ekonomi      ditargetkan tumbuh     persen  pa...   
2  survei litbang kompas  pdi p  gerindra  dan go...   
3  survei litbang  kompas   popularitas golkar te...   
4   endorsement  dan basa basi politik ala jokowi...   

                                         vector_cbow  \
0  [0.1068104, 0.16480927, -0.06170332, 0.4143358...   
1  [0.10346157, 0.15733077, -0.07548306, 0.380306...   
2  [0.101267934, 0.16018696, -0.06279436, 0.34137...   
3  [0.090119496, 0.16516396, -0.058607463, 0.3756...   
4  [0.09179839, 0.15346093, -0.053987127, 0.39101...   

                                           vector_sg  \
0  [0.028260801, -0.06512354, -0.109418556, 0.042...   
1  [0.032424506, -0.06527497, -0.10640011, 0.0455...   
2  [0.029482245, -0.0674338, -0.10877135, 0.04778...   
3  [0.028165009, -0.06870859, -0.10876783, 0.0469...   
4  [0.0276

In [ ]:
# Simpan DataFrame ke pickle
df_cleaned.to_pickle('combined_with_vectors_cbow_sg.pkl')

# Konversi vektor gabungan ke string untuk disimpan ke Excel
df_cleaned['vector_combined_str'] = df_cleaned['vector_combined'].apply(lambda x: ','.join(map(str, x)))

# Simpan DataFrame ke Excel (vektor gabungan dalam bentuk string)
df_cleaned.to_excel('combined_with_vectors_cbow_sg.xlsx', index=False)

print("\nDataFrame dengan vektor CBOW, Skip-gram, dan Gabungan telah disimpan.")

/tmp/ipython-input-679521417.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['vector_combined_str'] = df_cleaned['vector_combined'].apply(lambda x: ','.join(map(str, x)))



DataFrame dengan vektor CBOW, Skip-gram, dan Gabungan telah disimpan.
